# Notebook 1: Social Media Data Collection
## Retail Investor Sentiment from Reddit r/WallStreetBets

---

**Research Project:** Retail Sentiment, Earnings Quality, and Stock Returns

**Purpose:** Collect and structure social media posts from Reddit's r/WallStreetBets and related retail investor forums for the period 2018-2023.

**Data Sources:**
- Reddit API (PRAW)
- Pushshift.io Archives (Academic Reddit Data)
- BigQuery Reddit Dataset

**Output:** Post-level table with `[platform_post_id, ticker, date, time, text, score, author, subreddit]`

---

**Citation:** If using this code, please cite appropriately.

**Last Updated:** 2024

## 1. Environment Setup and Dependencies

In [ ]:
# =============================================================================
# INSTALL REQUIRED PACKAGES
# =============================================================================
# Run this cell once to install all dependencies

!pip install praw==7.7.1
!pip install pmaw==3.0.0
!pip install pandas==2.0.3
!pip install numpy==1.24.3
!pip install tqdm==4.66.1
!pip install requests==2.31.0
!pip install pyarrow==14.0.1
!pip install google-cloud-bigquery==3.13.0
!pip install db-dtypes==1.2.0

print("All packages installed successfully.")

In [ ]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

import os
import re
import json
import time
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Set, Optional, Tuple
from collections import defaultdict

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import requests

# Reddit API
import praw
from pmaw import PushshiftAPI

# Google BigQuery (for large-scale Reddit data)
from google.cloud import bigquery
from google.colab import auth

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print(f"Environment setup complete. Timestamp: {datetime.now()}")

## 2. Configuration and Parameters

### 2.1 Research Design Parameters

Following established literature on social media and asset pricing (e.g., Cookson & Niessner, 2020; Bradley et al., 2021), we define our sample parameters.

In [ ]:
# =============================================================================
# RESEARCH PARAMETERS
# =============================================================================

class ResearchConfig:
    """Configuration class for research parameters.
    
    This centralizes all parameters for reproducibility and sensitivity analysis.
    """
    
    # Sample Period
    START_DATE = "2018-01-01"
    END_DATE = "2023-12-31"
    
    # Target Subreddits (Primary and Extension)
    PRIMARY_SUBREDDITS = ["wallstreetbets"]
    EXTENSION_SUBREDDITS = ["stocks", "investing", "options"]  # Optional
    
    # Minimum thresholds for data quality
    MIN_POST_LENGTH = 10  # Characters
    MIN_SCORE_THRESHOLD = -100  # Include downvoted but not spam
    
    # Data storage paths (Google Drive mount)
    BASE_PATH = "/content/drive/MyDrive/Research/RetailSentiment/"
    RAW_DATA_PATH = BASE_PATH + "data/raw/"
    PROCESSED_DATA_PATH = BASE_PATH + "data/processed/"
    
    # API Rate Limits
    REDDIT_RATE_LIMIT = 60  # Requests per minute
    SLEEP_BETWEEN_REQUESTS = 1.0  # Seconds
    
    @classmethod
    def print_config(cls):
        """Print current configuration for documentation."""
        print("="*60)
        print("RESEARCH CONFIGURATION")
        print("="*60)
        print(f"Sample Period: {cls.START_DATE} to {cls.END_DATE}")
        print(f"Primary Subreddits: {cls.PRIMARY_SUBREDDITS}")
        print(f"Extension Subreddits: {cls.EXTENSION_SUBREDDITS}")
        print(f"Minimum Post Length: {cls.MIN_POST_LENGTH} chars")
        print("="*60)

config = ResearchConfig()
config.print_config()

### 2.2 Mount Google Drive for Data Persistence

In [ ]:
# =============================================================================
# MOUNT GOOGLE DRIVE
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

# Create directory structure
os.makedirs(config.RAW_DATA_PATH, exist_ok=True)
os.makedirs(config.PROCESSED_DATA_PATH, exist_ok=True)

print(f"Data directories created at: {config.BASE_PATH}")

## 3. Ticker Universe Construction

### 3.1 Build Comprehensive Ticker Dictionary

We construct a ticker universe based on S&P 500 and Russell 1000 constituents, with careful handling of:
- Ambiguous tickers (common words)
- Ticker changes over time
- Multiple share classes

In [ ]:
# =============================================================================
# TICKER UNIVERSE CONSTRUCTION
# =============================================================================

class TickerUniverse:
    """Manages the ticker universe for text-to-ticker mapping.
    
    Following Cookson, Engelberg, and Mullins (2020), we carefully construct
    a ticker dictionary that excludes ambiguous symbols.
    """
    
    # Ambiguous tickers to exclude (common words/abbreviations)
    AMBIGUOUS_TICKERS = {
        'IT', 'ALL', 'A', 'AN', 'ARE', 'AT', 'BE', 'BY', 'FOR', 'HAS',
        'HE', 'HIM', 'HIS', 'HOW', 'IF', 'IN', 'IS', 'IT', 'ITS', 'ME',
        'MY', 'NEW', 'NOW', 'OF', 'ON', 'ONE', 'OR', 'OUR', 'OUT', 'SO',
        'THE', 'TO', 'UP', 'US', 'WAS', 'WE', 'WHO', 'YOU', 'DD', 'TD',
        'CEO', 'CFO', 'COO', 'IPO', 'ATH', 'ATL', 'EOD', 'EPS', 'ETF',
        'GDP', 'IMO', 'LOL', 'NYC', 'USA', 'WSB', 'YOLO', 'FD', 'TA',
        'IV', 'OI', 'PM', 'AM', 'PT', 'ER', 'PE', 'PS', 'PB', 'EV',
        'AI', 'AR', 'VR', 'UK', 'EU', 'UN', 'FBI', 'SEC', 'FED', 'GO',
        'LOVE', 'LIFE', 'REAL', 'TRUE', 'WELL', 'GOOD', 'BEST', 'NEXT',
        'VERY', 'JUST', 'LIKE', 'EVEN', 'ONLY', 'OVER', 'SUCH', 'MOST',
        'BEEN', 'WERE', 'WHAT', 'WHEN', 'CASH', 'GAIN', 'LOSS', 'HOLD',
        'PUMP', 'DUMP', 'MOON', 'BEAR', 'BULL', 'CALL', 'POST', 'EDIT',
        'INFO', 'TECH', 'FUND', 'RISK', 'GOLD', 'OPEN', 'SAVE', 'FAST',
        'HUGE', 'PEAK', 'CARE', 'HOPE', 'STAY', 'PLAY', 'BIG', 'LOW',
        'HIGH', 'AWAY', 'DONT', 'CANT', 'WONT', 'FREE', 'EASY', 'HARD'
    }
    
    # WSB-specific slang that might be confused with tickers
    WSB_SLANG = {
        'YOLO', 'FOMO', 'HODL', 'ROPE', 'MOON', 'APES', 'WIFE',
        'FANS', 'GANG', 'TEAM', 'KING', 'BABY', 'RICH', 'POOR'
    }
    
    def __init__(self):
        self.tickers = {}
        self.ticker_to_company = {}
        self.excluded = self.AMBIGUOUS_TICKERS.union(self.WSB_SLANG)
        
    def load_sp500(self) -> pd.DataFrame:
        """Load current S&P 500 constituents from Wikipedia."""
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        sp500 = tables[0]
        return sp500[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
    
    def load_russell1000(self) -> pd.DataFrame:
        """Load Russell 1000 constituents.
        
        Note: Russell 1000 list requires alternative source.
        We use a combination of sources for comprehensive coverage.
        """
        # Alternative: Load from saved file or API
        # For now, we extend S&P 500 with additional large caps
        pass
    
    def build_universe(self) -> Dict[str, str]:
        """Build the complete ticker universe.
        
        Returns:
            Dictionary mapping ticker to company name
        """
        print("Building ticker universe...")
        
        # Load S&P 500
        try:
            sp500 = self.load_sp500()
            print(f"Loaded {len(sp500)} S&P 500 constituents")
        except Exception as e:
            print(f"Warning: Could not load S&P 500 from Wikipedia: {e}")
            sp500 = pd.DataFrame(columns=['Symbol', 'Security'])
        
        # Process tickers
        for _, row in sp500.iterrows():
            ticker = row['Symbol'].replace('.', '-').upper().strip()
            if ticker not in self.excluded and len(ticker) >= 1:
                self.ticker_to_company[ticker] = row['Security']
        
        # Add major stocks that might be missing
        additional_tickers = {
            'GME': 'GameStop Corp',
            'AMC': 'AMC Entertainment',
            'BB': 'BlackBerry Limited',
            'NOK': 'Nokia Corporation',
            'PLTR': 'Palantir Technologies',
            'SPCE': 'Virgin Galactic',
            'WISH': 'ContextLogic Inc',
            'CLOV': 'Clover Health',
            'SOFI': 'SoFi Technologies',
            'RIVN': 'Rivian Automotive',
            'LCID': 'Lucid Group'
        }
        
        for ticker, company in additional_tickers.items():
            if ticker not in self.excluded:
                self.ticker_to_company[ticker] = company
        
        print(f"Final universe: {len(self.ticker_to_company)} tickers")
        print(f"Excluded ambiguous: {len(self.excluded)} symbols")
        
        return self.ticker_to_company
    
    def get_regex_pattern(self) -> str:
        """Generate regex pattern for ticker extraction.
        
        Pattern matches:
        - $TICKER format (most reliable)
        - Standalone TICKER with word boundaries
        """
        tickers = list(self.ticker_to_company.keys())
        # Sort by length (longest first) to avoid partial matches
        tickers_sorted = sorted(tickers, key=len, reverse=True)
        
        # Pattern: $TICKER or word-boundary TICKER
        pattern = r'(?:\$(' + '|'.join(tickers_sorted) + r')\b|\b(' + '|'.join(tickers_sorted) + r')\b)'
        return pattern

# Initialize ticker universe
ticker_universe = TickerUniverse()
ticker_dict = ticker_universe.build_universe()

In [ ]:
# =============================================================================
# TICKER EXTRACTION FUNCTION
# =============================================================================

class TickerExtractor:
    """Extracts stock tickers from social media text.
    
    Methodology follows Cookson & Niessner (2020) and Bradley et al. (2021):
    1. Prefer $TICKER cashtag format (highest precision)
    2. Fall back to standalone ticker with context validation
    3. Return primary ticker (first mentioned in title, then body)
    """
    
    def __init__(self, ticker_dict: Dict[str, str]):
        self.ticker_dict = ticker_dict
        self.valid_tickers = set(ticker_dict.keys())
        
        # Compile regex patterns
        tickers_sorted = sorted(self.valid_tickers, key=len, reverse=True)
        
        # Cashtag pattern: $TICKER
        self.cashtag_pattern = re.compile(
            r'\$(' + '|'.join(tickers_sorted) + r')\b',
            re.IGNORECASE
        )
        
        # Standalone pattern: TICKER (all caps, word boundary)
        self.standalone_pattern = re.compile(
            r'\b(' + '|'.join(tickers_sorted) + r')\b'
        )
        
    def extract_tickers(self, text: str, prefer_cashtag: bool = True) -> List[str]:
        """Extract all valid tickers from text.
        
        Args:
            text: Input text (title or body)
            prefer_cashtag: If True, prioritize $TICKER matches
            
        Returns:
            List of extracted tickers in order of appearance
        """
        if not text or not isinstance(text, str):
            return []
        
        tickers_found = []
        positions = {}
        
        # Extract cashtags first (highest confidence)
        for match in self.cashtag_pattern.finditer(text):
            ticker = match.group(1).upper()
            if ticker in self.valid_tickers:
                pos = match.start()
                if ticker not in positions or pos < positions[ticker]:
                    positions[ticker] = pos
        
        # Extract standalone tickers (lower confidence)
        for match in self.standalone_pattern.finditer(text):
            ticker = match.group(1).upper()
            if ticker in self.valid_tickers and ticker not in positions:
                # Only add if ticker is in ALL CAPS in original text
                original = text[match.start():match.end()]
                if original.isupper():
                    positions[ticker] = match.start()
        
        # Sort by position
        tickers_found = sorted(positions.keys(), key=lambda x: positions[x])
        return tickers_found
    
    def get_primary_ticker(self, title: str, body: str) -> Optional[str]:
        """Get primary ticker for a post.
        
        Priority:
        1. First cashtag in title
        2. First standalone ticker in title
        3. First cashtag in body
        4. First standalone ticker in body
        
        Returns:
            Primary ticker or None if no valid ticker found
        """
        # Check title first
        title_tickers = self.extract_tickers(title or '')
        if title_tickers:
            return title_tickers[0]
        
        # Fall back to body
        body_tickers = self.extract_tickers(body or '')
        if body_tickers:
            return body_tickers[0]
        
        return None
    
    def get_all_tickers(self, title: str, body: str) -> List[str]:
        """Get all unique tickers mentioned in a post."""
        title_tickers = self.extract_tickers(title or '')
        body_tickers = self.extract_tickers(body or '')
        
        # Combine while preserving order
        all_tickers = []
        seen = set()
        for ticker in title_tickers + body_tickers:
            if ticker not in seen:
                all_tickers.append(ticker)
                seen.add(ticker)
        
        return all_tickers

# Initialize extractor
ticker_extractor = TickerExtractor(ticker_dict)

# Test extraction
test_cases = [
    ("$GME to the moon! 🚀🚀🚀", "GameStop is going to squeeze"),
    ("AAPL earnings play", "Thinking of buying $AAPL calls before earnings"),
    ("My DD on Tesla", "TSLA is undervalued because of the energy business")
]

print("Testing ticker extraction:")
for title, body in test_cases:
    primary = ticker_extractor.get_primary_ticker(title, body)
    all_tickers = ticker_extractor.get_all_tickers(title, body)
    print(f"  Title: '{title}'")
    print(f"  Primary: {primary}, All: {all_tickers}\n")

## 4. Reddit API Configuration

### 4.1 Setup Reddit API Credentials

You need to create a Reddit application at https://www.reddit.com/prefs/apps to obtain credentials.

In [ ]:
# =============================================================================
# REDDIT API CREDENTIALS
# =============================================================================
# 
# IMPORTANT: Store credentials securely. Do not commit to version control.
#
# To obtain credentials:
# 1. Go to https://www.reddit.com/prefs/apps
# 2. Click "create another app"
# 3. Select "script" as the application type
# 4. Fill in name and redirect URI (http://localhost:8080)
# 5. Copy client_id (under app name) and client_secret

# Option 1: Enter credentials manually (will prompt)
from getpass import getpass

REDDIT_CLIENT_ID = getpass("Enter Reddit Client ID: ")
REDDIT_CLIENT_SECRET = getpass("Enter Reddit Client Secret: ")
REDDIT_USER_AGENT = "ResearchBot/1.0 (Academic Research on Retail Sentiment)"

print("Credentials stored (not displayed for security).")

In [ ]:
# =============================================================================
# INITIALIZE REDDIT API CLIENT
# =============================================================================

def initialize_reddit_client(client_id: str, client_secret: str, 
                              user_agent: str) -> praw.Reddit:
    """Initialize and validate Reddit API connection.
    
    Args:
        client_id: Reddit API client ID
        client_secret: Reddit API client secret
        user_agent: Descriptive user agent string
        
    Returns:
        Authenticated Reddit instance
    """
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )
    
    # Validate connection
    try:
        # Test API access
        reddit.user.me()  # Will be None for script apps, but validates credentials
        print("Reddit API connection successful (read-only mode)")
    except Exception as e:
        print(f"Warning: Could not fully validate API access: {e}")
        print("Continuing with read-only public access...")
    
    return reddit

# Initialize client
reddit = initialize_reddit_client(
    REDDIT_CLIENT_ID,
    REDDIT_CLIENT_SECRET,
    REDDIT_USER_AGENT
)

## 5. Data Collection Methods

### 5.1 Method A: Pushshift API (Historical Data)

Pushshift.io provides comprehensive historical Reddit data. This is the preferred method for academic research.

In [ ]:
# =============================================================================
# PUSHSHIFT DATA COLLECTOR
# =============================================================================

class PushshiftCollector:
    """Collects historical Reddit data using Pushshift API.
    
    Pushshift provides access to historical Reddit data that is no longer
    available through the official Reddit API.
    
    Note: Pushshift availability has varied. Check current status at:
    https://pushshift.io/
    """
    
    def __init__(self, reddit_client: praw.Reddit = None):
        """Initialize Pushshift API client.
        
        Args:
            reddit_client: Optional PRAW client for enriching data
        """
        self.api = PushshiftAPI(praw=reddit_client)
        self.reddit = reddit_client
        
    def collect_submissions(
        self,
        subreddit: str,
        start_date: str,
        end_date: str,
        limit: int = None
    ) -> pd.DataFrame:
        """Collect submissions (posts) from a subreddit.
        
        Args:
            subreddit: Subreddit name (without r/)
            start_date: Start date (YYYY-MM-DD)
            end_date: End date (YYYY-MM-DD)
            limit: Maximum number of submissions (None for all)
            
        Returns:
            DataFrame with submission data
        """
        # Convert dates to timestamps
        start_ts = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
        end_ts = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
        
        print(f"Collecting submissions from r/{subreddit}")
        print(f"Period: {start_date} to {end_date}")
        
        # Query Pushshift
        submissions = self.api.search_submissions(
            subreddit=subreddit,
            after=start_ts,
            before=end_ts,
            limit=limit,
            filter=['id', 'created_utc', 'title', 'selftext', 'score',
                    'num_comments', 'author', 'permalink', 'upvote_ratio']
        )
        
        # Convert to list
        posts_list = []
        for post in tqdm(submissions, desc="Processing submissions"):
            posts_list.append({
                'post_id': post.get('id'),
                'created_utc': post.get('created_utc'),
                'title': post.get('title'),
                'selftext': post.get('selftext'),
                'score': post.get('score'),
                'num_comments': post.get('num_comments'),
                'author': str(post.get('author')),
                'permalink': post.get('permalink'),
                'upvote_ratio': post.get('upvote_ratio'),
                'subreddit': subreddit
            })
        
        df = pd.DataFrame(posts_list)
        
        if len(df) > 0:
            # Convert timestamp to datetime
            df['datetime'] = pd.to_datetime(df['created_utc'], unit='s')
            df['date'] = df['datetime'].dt.date
            df['time'] = df['datetime'].dt.time
            
            print(f"Collected {len(df):,} submissions")
        else:
            print("No submissions found")
            
        return df
    
    def collect_comments(
        self,
        subreddit: str,
        start_date: str,
        end_date: str,
        limit: int = None
    ) -> pd.DataFrame:
        """Collect comments from a subreddit.
        
        Args:
            subreddit: Subreddit name (without r/)
            start_date: Start date (YYYY-MM-DD)
            end_date: End date (YYYY-MM-DD)
            limit: Maximum number of comments (None for all)
            
        Returns:
            DataFrame with comment data
        """
        start_ts = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
        end_ts = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
        
        print(f"Collecting comments from r/{subreddit}")
        print(f"Period: {start_date} to {end_date}")
        
        comments = self.api.search_comments(
            subreddit=subreddit,
            after=start_ts,
            before=end_ts,
            limit=limit,
            filter=['id', 'created_utc', 'body', 'score', 'author',
                    'link_id', 'parent_id', 'permalink']
        )
        
        comments_list = []
        for comment in tqdm(comments, desc="Processing comments"):
            comments_list.append({
                'comment_id': comment.get('id'),
                'created_utc': comment.get('created_utc'),
                'body': comment.get('body'),
                'score': comment.get('score'),
                'author': str(comment.get('author')),
                'link_id': comment.get('link_id'),
                'parent_id': comment.get('parent_id'),
                'permalink': comment.get('permalink'),
                'subreddit': subreddit
            })
        
        df = pd.DataFrame(comments_list)
        
        if len(df) > 0:
            df['datetime'] = pd.to_datetime(df['created_utc'], unit='s')
            df['date'] = df['datetime'].dt.date
            df['time'] = df['datetime'].dt.time
            
            print(f"Collected {len(df):,} comments")
        else:
            print("No comments found")
            
        return df

# Initialize collector
pushshift_collector = PushshiftCollector(reddit)
print("Pushshift collector initialized")

### 5.2 Method B: BigQuery Reddit Dataset

Google BigQuery hosts historical Reddit data. This is efficient for large-scale queries.

In [ ]:
# =============================================================================
# BIGQUERY REDDIT COLLECTOR
# =============================================================================

class BigQueryRedditCollector:
    """Collects Reddit data from Google BigQuery public datasets.
    
    The fh-bigquery:reddit datasets contain historical Reddit posts and
    comments. This is efficient for large-scale academic research.
    
    Note: Requires Google Cloud authentication.
    """
    
    def __init__(self, project_id: str = None):
        """Initialize BigQuery client.
        
        Args:
            project_id: Google Cloud project ID
        """
        # Authenticate with Google Cloud
        auth.authenticate_user()
        
        self.client = bigquery.Client(project=project_id)
        self.project_id = project_id
        print("BigQuery client initialized")
        
    def query_submissions(
        self,
        subreddits: List[str],
        start_date: str,
        end_date: str,
        min_score: int = -100
    ) -> pd.DataFrame:
        """Query Reddit submissions from BigQuery.
        
        Args:
            subreddits: List of subreddit names
            start_date: Start date (YYYY-MM-DD)
            end_date: End date (YYYY-MM-DD)
            min_score: Minimum post score filter
            
        Returns:
            DataFrame with submission data
        """
        subreddit_list = "','".join(subreddits)
        
        query = f"""
        SELECT
            id AS post_id,
            created_utc,
            TIMESTAMP_SECONDS(created_utc) AS datetime,
            DATE(TIMESTAMP_SECONDS(created_utc)) AS date,
            title,
            selftext,
            score,
            num_comments,
            author,
            subreddit,
            permalink,
            upvote_ratio
        FROM `fh-bigquery.reddit_posts.*`
        WHERE subreddit IN ('{subreddit_list}')
            AND DATE(TIMESTAMP_SECONDS(created_utc)) >= '{start_date}'
            AND DATE(TIMESTAMP_SECONDS(created_utc)) <= '{end_date}'
            AND score >= {min_score}
        ORDER BY created_utc
        """
        
        print(f"Querying BigQuery for submissions...")
        print(f"Subreddits: {subreddits}")
        print(f"Period: {start_date} to {end_date}")
        
        df = self.client.query(query).to_dataframe()
        print(f"Retrieved {len(df):,} submissions")
        
        return df
    
    def query_comments(
        self,
        subreddits: List[str],
        start_date: str,
        end_date: str,
        min_score: int = -100
    ) -> pd.DataFrame:
        """Query Reddit comments from BigQuery.
        
        Args:
            subreddits: List of subreddit names
            start_date: Start date (YYYY-MM-DD)
            end_date: End date (YYYY-MM-DD)
            min_score: Minimum comment score filter
            
        Returns:
            DataFrame with comment data
        """
        subreddit_list = "','".join(subreddits)
        
        query = f"""
        SELECT
            id AS comment_id,
            created_utc,
            TIMESTAMP_SECONDS(created_utc) AS datetime,
            DATE(TIMESTAMP_SECONDS(created_utc)) AS date,
            body,
            score,
            author,
            subreddit,
            link_id,
            parent_id
        FROM `fh-bigquery.reddit_comments.*`
        WHERE subreddit IN ('{subreddit_list}')
            AND DATE(TIMESTAMP_SECONDS(created_utc)) >= '{start_date}'
            AND DATE(TIMESTAMP_SECONDS(created_utc)) <= '{end_date}'
            AND score >= {min_score}
        ORDER BY created_utc
        """
        
        print(f"Querying BigQuery for comments...")
        df = self.client.query(query).to_dataframe()
        print(f"Retrieved {len(df):,} comments")
        
        return df

# Note: Initialize only when needed due to authentication requirement
# bq_collector = BigQueryRedditCollector(project_id="your-project-id")
print("BigQuery collector class defined (initialize when ready)")

### 5.3 Method C: Pre-Existing Academic Datasets

For reproducibility, we can also use pre-existing Reddit datasets (e.g., from Kaggle or academic archives).

In [ ]:
# =============================================================================
# KAGGLE / ACADEMIC DATASET LOADER
# =============================================================================

class ExternalDatasetLoader:
    """Loads pre-existing Reddit datasets from external sources.
    
    Sources:
    - Kaggle WSB datasets
    - Academic Reddit archives
    - Pushshift data dumps
    """
    
    # Known Kaggle datasets for WSB
    KAGGLE_DATASETS = {
        'wsb_daily': 'unanimad/reddit-rwallstreetbets',
        'wsb_2021': 'mattpodolak/rwallstreetbets-posts-and-comments',
        'gme_saga': 'gpreda/reddit-wsb-gme-gamestop'
    }
    
    def __init__(self, data_dir: str):
        """Initialize loader with data directory.
        
        Args:
            data_dir: Directory to store downloaded data
        """
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        
    def setup_kaggle(self):
        """Setup Kaggle API authentication.
        
        Requires kaggle.json file with API credentials.
        """
        # Install Kaggle if needed
        try:
            import kaggle
        except ImportError:
            os.system('pip install kaggle')
            import kaggle
        
        print("Kaggle API ready")
        print("Note: Place kaggle.json in ~/.kaggle/ for authentication")
        
    def download_kaggle_dataset(self, dataset_key: str) -> str:
        """Download a dataset from Kaggle.
        
        Args:
            dataset_key: Key from KAGGLE_DATASETS or full dataset path
            
        Returns:
            Path to downloaded data
        """
        dataset_path = self.KAGGLE_DATASETS.get(dataset_key, dataset_key)
        
        output_dir = os.path.join(self.data_dir, dataset_key)
        os.makedirs(output_dir, exist_ok=True)
        
        os.system(f'kaggle datasets download -d {dataset_path} -p {output_dir} --unzip')
        
        print(f"Dataset downloaded to: {output_dir}")
        return output_dir
    
    def load_csv_dataset(self, filepath: str) -> pd.DataFrame:
        """Load a CSV dataset with standard preprocessing.
        
        Args:
            filepath: Path to CSV file
            
        Returns:
            Preprocessed DataFrame
        """
        print(f"Loading: {filepath}")
        
        # Try different encodings
        for encoding in ['utf-8', 'latin-1', 'cp1252']:
            try:
                df = pd.read_csv(filepath, encoding=encoding, low_memory=False)
                break
            except UnicodeDecodeError:
                continue
        
        print(f"Loaded {len(df):,} rows")
        print(f"Columns: {list(df.columns)}")
        
        return df
    
    def standardize_columns(self, df: pd.DataFrame, 
                           column_mapping: Dict[str, str]) -> pd.DataFrame:
        """Standardize column names across different datasets.
        
        Args:
            df: Input DataFrame
            column_mapping: Mapping from original to standard names
            
        Returns:
            DataFrame with standardized columns
        """
        df_renamed = df.rename(columns=column_mapping)
        return df_renamed

# Initialize loader
external_loader = ExternalDatasetLoader(config.RAW_DATA_PATH)
print("External dataset loader initialized")

## 6. Data Collection Execution

### 6.1 Collect WSB Data by Year

We collect data in yearly chunks to manage memory and enable checkpointing.

In [ ]:
# =============================================================================
# DATA COLLECTION PIPELINE
# =============================================================================

class DataCollectionPipeline:
    """Orchestrates the data collection process.
    
    Features:
    - Yearly chunked collection for memory management
    - Checkpoint saving for interruption recovery
    - Multiple source fallback
    - Progress tracking
    """
    
    def __init__(self, config: ResearchConfig, ticker_extractor: TickerExtractor):
        self.config = config
        self.ticker_extractor = ticker_extractor
        self.collected_data = {}
        
    def collect_year(
        self,
        year: int,
        subreddits: List[str],
        collector: PushshiftCollector,
        collect_comments: bool = True
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Collect data for a specific year.
        
        Args:
            year: Year to collect
            subreddits: List of subreddits
            collector: Data collector instance
            collect_comments: Whether to collect comments
            
        Returns:
            Tuple of (submissions_df, comments_df)
        """
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        
        print(f"\n{'='*60}")
        print(f"Collecting data for {year}")
        print(f"{'='*60}")
        
        all_submissions = []
        all_comments = []
        
        for subreddit in subreddits:
            print(f"\nProcessing r/{subreddit}...")
            
            # Collect submissions
            try:
                submissions = collector.collect_submissions(
                    subreddit=subreddit,
                    start_date=start_date,
                    end_date=end_date
                )
                all_submissions.append(submissions)
            except Exception as e:
                print(f"Error collecting submissions: {e}")
            
            # Collect comments (optional)
            if collect_comments:
                try:
                    comments = collector.collect_comments(
                        subreddit=subreddit,
                        start_date=start_date,
                        end_date=end_date
                    )
                    all_comments.append(comments)
                except Exception as e:
                    print(f"Error collecting comments: {e}")
            
            # Rate limiting
            time.sleep(self.config.SLEEP_BETWEEN_REQUESTS)
        
        # Combine DataFrames
        submissions_df = pd.concat(all_submissions, ignore_index=True) if all_submissions else pd.DataFrame()
        comments_df = pd.concat(all_comments, ignore_index=True) if all_comments else pd.DataFrame()
        
        return submissions_df, comments_df
    
    def add_ticker_mapping(self, df: pd.DataFrame, 
                          text_col: str, title_col: str = None) -> pd.DataFrame:
        """Add ticker mapping to DataFrame.
        
        Args:
            df: Input DataFrame
            text_col: Column containing main text (body/selftext)
            title_col: Column containing title (optional)
            
        Returns:
            DataFrame with ticker columns added
        """
        print("Extracting tickers...")
        
        tickers_primary = []
        tickers_all = []
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Ticker extraction"):
            title = row.get(title_col, '') if title_col else ''
            text = row.get(text_col, '') or ''
            
            primary = self.ticker_extractor.get_primary_ticker(title, text)
            all_tickers = self.ticker_extractor.get_all_tickers(title, text)
            
            tickers_primary.append(primary)
            tickers_all.append(all_tickers)
        
        df['ticker_primary'] = tickers_primary
        df['tickers_all'] = tickers_all
        df['ticker_count'] = df['tickers_all'].apply(len)
        
        # Statistics
        has_ticker = df['ticker_primary'].notna().sum()
        print(f"Posts with identified ticker: {has_ticker:,} ({100*has_ticker/len(df):.1f}%)")
        
        return df
    
    def save_checkpoint(self, df: pd.DataFrame, name: str, year: int):
        """Save data checkpoint to disk.
        
        Args:
            df: DataFrame to save
            name: Dataset name (submissions/comments)
            year: Year of data
        """
        filepath = os.path.join(
            self.config.RAW_DATA_PATH,
            f"wsb_{name}_{year}.parquet"
        )
        df.to_parquet(filepath, index=False)
        print(f"Saved checkpoint: {filepath}")
        
    def load_checkpoint(self, name: str, year: int) -> Optional[pd.DataFrame]:
        """Load data checkpoint from disk.
        
        Args:
            name: Dataset name
            year: Year of data
            
        Returns:
            DataFrame or None if not found
        """
        filepath = os.path.join(
            self.config.RAW_DATA_PATH,
            f"wsb_{name}_{year}.parquet"
        )
        if os.path.exists(filepath):
            print(f"Loading checkpoint: {filepath}")
            return pd.read_parquet(filepath)
        return None

# Initialize pipeline
pipeline = DataCollectionPipeline(config, ticker_extractor)
print("Data collection pipeline initialized")

In [ ]:
# =============================================================================
# EXECUTE DATA COLLECTION
# =============================================================================

# Define collection years
YEARS_TO_COLLECT = [2018, 2019, 2020, 2021, 2022, 2023]

# Storage for all collected data
all_submissions = []
all_comments = []

# Collect data year by year
for year in YEARS_TO_COLLECT:
    print(f"\n{'#'*60}")
    print(f"# COLLECTING DATA FOR {year}")
    print(f"{'#'*60}")
    
    # Check for existing checkpoint
    submissions_checkpoint = pipeline.load_checkpoint('submissions', year)
    comments_checkpoint = pipeline.load_checkpoint('comments', year)
    
    if submissions_checkpoint is not None:
        print(f"Using cached submissions for {year}")
        year_submissions = submissions_checkpoint
    else:
        # Collect new data
        year_submissions, year_comments = pipeline.collect_year(
            year=year,
            subreddits=config.PRIMARY_SUBREDDITS,
            collector=pushshift_collector,
            collect_comments=True
        )
        
        # Add ticker mapping
        if len(year_submissions) > 0:
            year_submissions = pipeline.add_ticker_mapping(
                year_submissions, 
                text_col='selftext', 
                title_col='title'
            )
            pipeline.save_checkpoint(year_submissions, 'submissions', year)
        
        if len(year_comments) > 0:
            year_comments = pipeline.add_ticker_mapping(
                year_comments,
                text_col='body'
            )
            pipeline.save_checkpoint(year_comments, 'comments', year)
            all_comments.append(year_comments)
    
    if year_submissions is not None and len(year_submissions) > 0:
        all_submissions.append(year_submissions)

print("\n" + "="*60)
print("DATA COLLECTION COMPLETE")
print("="*60)

## 7. Data Quality and Validation

### 7.1 Data Quality Checks

In [ ]:
# =============================================================================
# DATA QUALITY VALIDATION
# =============================================================================

class DataQualityValidator:
    """Validates data quality for academic research standards.
    
    Following best practices from top finance journals.
    """
    
    def __init__(self):
        self.validation_results = {}
        
    def validate_submissions(self, df: pd.DataFrame) -> Dict:
        """Run comprehensive validation on submissions data.
        
        Args:
            df: Submissions DataFrame
            
        Returns:
            Dictionary of validation results
        """
        results = {
            'total_records': len(df),
            'date_range': (df['date'].min(), df['date'].max()),
            'missing_values': {},
            'ticker_coverage': {},
            'temporal_distribution': {},
            'quality_flags': []
        }
        
        # Missing values check
        for col in df.columns:
            missing = df[col].isna().sum()
            if missing > 0:
                results['missing_values'][col] = {
                    'count': missing,
                    'pct': 100 * missing / len(df)
                }
        
        # Ticker coverage
        if 'ticker_primary' in df.columns:
            has_ticker = df['ticker_primary'].notna().sum()
            results['ticker_coverage'] = {
                'posts_with_ticker': has_ticker,
                'coverage_pct': 100 * has_ticker / len(df),
                'unique_tickers': df['ticker_primary'].nunique(),
                'top_tickers': df['ticker_primary'].value_counts().head(20).to_dict()
            }
        
        # Temporal distribution
        if 'date' in df.columns:
            daily_counts = df.groupby('date').size()
            results['temporal_distribution'] = {
                'days_covered': len(daily_counts),
                'avg_posts_per_day': daily_counts.mean(),
                'min_posts_day': daily_counts.min(),
                'max_posts_day': daily_counts.max(),
                'days_with_zero_posts': (daily_counts == 0).sum()
            }
        
        # Quality flags
        if results['ticker_coverage'].get('coverage_pct', 0) < 10:
            results['quality_flags'].append('LOW_TICKER_COVERAGE')
        
        if len(df) < 10000:
            results['quality_flags'].append('SMALL_SAMPLE')
            
        return results
    
    def print_validation_report(self, results: Dict):
        """Print formatted validation report."""
        print("\n" + "="*60)
        print("DATA QUALITY VALIDATION REPORT")
        print("="*60)
        
        print(f"\nTotal Records: {results['total_records']:,}")
        print(f"Date Range: {results['date_range'][0]} to {results['date_range'][1]}")
        
        print("\n--- Missing Values ---")
        if results['missing_values']:
            for col, info in results['missing_values'].items():
                print(f"  {col}: {info['count']:,} ({info['pct']:.2f}%)")
        else:
            print("  No missing values")
        
        print("\n--- Ticker Coverage ---")
        tc = results['ticker_coverage']
        if tc:
            print(f"  Posts with ticker: {tc['posts_with_ticker']:,} ({tc['coverage_pct']:.1f}%)")
            print(f"  Unique tickers: {tc['unique_tickers']:,}")
            print(f"  Top 10 tickers:")
            for ticker, count in list(tc['top_tickers'].items())[:10]:
                print(f"    {ticker}: {count:,}")
        
        print("\n--- Temporal Distribution ---")
        td = results['temporal_distribution']
        if td:
            print(f"  Days covered: {td['days_covered']:,}")
            print(f"  Avg posts/day: {td['avg_posts_per_day']:.1f}")
            print(f"  Range: {td['min_posts_day']} - {td['max_posts_day']}")
        
        print("\n--- Quality Flags ---")
        if results['quality_flags']:
            for flag in results['quality_flags']:
                print(f"  ⚠️  {flag}")
        else:
            print("  ✓ No quality issues detected")
        
        print("\n" + "="*60)

# Initialize validator
validator = DataQualityValidator()

In [ ]:
# =============================================================================
# RUN VALIDATION ON COLLECTED DATA
# =============================================================================

# Combine all submissions
if all_submissions:
    combined_submissions = pd.concat(all_submissions, ignore_index=True)
    print(f"Combined submissions: {len(combined_submissions):,} records")
    
    # Run validation
    validation_results = validator.validate_submissions(combined_submissions)
    validator.print_validation_report(validation_results)
else:
    print("No submissions data collected yet.")

## 8. Output: Post-Level Table

### 8.1 Create Final Output Table

In [ ]:
# =============================================================================
# CREATE FINAL OUTPUT TABLE
# =============================================================================

def create_post_level_table(submissions_df: pd.DataFrame, 
                            comments_df: pd.DataFrame = None) -> pd.DataFrame:
    """Create the final post-level table for analysis.
    
    Output schema:
    [platform_post_id, ticker, date, time, text, score, author, subreddit]
    
    Args:
        submissions_df: Submissions data
        comments_df: Comments data (optional)
        
    Returns:
        Combined post-level DataFrame
    """
    posts = []
    
    # Process submissions
    if submissions_df is not None and len(submissions_df) > 0:
        submissions_processed = pd.DataFrame({
            'platform_post_id': 'submission_' + submissions_df['post_id'].astype(str),
            'ticker': submissions_df['ticker_primary'],
            'date': pd.to_datetime(submissions_df['date']),
            'time': submissions_df['time'],
            'datetime': submissions_df['datetime'],
            'title': submissions_df['title'],
            'text': submissions_df['selftext'],
            'score': submissions_df['score'],
            'num_comments': submissions_df['num_comments'],
            'author': submissions_df['author'],
            'subreddit': submissions_df['subreddit'],
            'tickers_all': submissions_df['tickers_all'],
            'ticker_count': submissions_df['ticker_count'],
            'post_type': 'submission'
        })
        posts.append(submissions_processed)
        print(f"Processed {len(submissions_processed):,} submissions")
    
    # Process comments
    if comments_df is not None and len(comments_df) > 0:
        comments_processed = pd.DataFrame({
            'platform_post_id': 'comment_' + comments_df['comment_id'].astype(str),
            'ticker': comments_df['ticker_primary'],
            'date': pd.to_datetime(comments_df['date']),
            'time': comments_df['time'],
            'datetime': comments_df['datetime'],
            'title': None,  # Comments don't have titles
            'text': comments_df['body'],
            'score': comments_df['score'],
            'num_comments': 0,
            'author': comments_df['author'],
            'subreddit': comments_df['subreddit'],
            'tickers_all': comments_df['tickers_all'],
            'ticker_count': comments_df['ticker_count'],
            'post_type': 'comment'
        })
        posts.append(comments_processed)
        print(f"Processed {len(comments_processed):,} comments")
    
    # Combine
    if posts:
        final_df = pd.concat(posts, ignore_index=True)
        
        # Sort by datetime
        final_df = final_df.sort_values('datetime').reset_index(drop=True)
        
        print(f"\nFinal post-level table: {len(final_df):,} records")
        return final_df
    else:
        print("No data to process")
        return pd.DataFrame()

# Create final table (uncomment when data is ready)
# post_level_table = create_post_level_table(combined_submissions, combined_comments)

In [ ]:
# =============================================================================
# SAVE FINAL OUTPUT
# =============================================================================

def save_post_level_table(df: pd.DataFrame, output_dir: str):
    """Save post-level table in multiple formats.
    
    Saves:
    - Parquet (efficient for analysis)
    - CSV (for compatibility)
    - Data dictionary (documentation)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Parquet (primary format)
    parquet_path = os.path.join(output_dir, 'wsb_posts_raw.parquet')
    df.to_parquet(parquet_path, index=False)
    print(f"Saved Parquet: {parquet_path}")
    
    # CSV (subset for inspection)
    csv_path = os.path.join(output_dir, 'wsb_posts_sample.csv')
    df.head(10000).to_csv(csv_path, index=False)
    print(f"Saved CSV sample: {csv_path}")
    
    # Data dictionary
    data_dict = {
        'platform_post_id': 'Unique identifier for post (submission_ or comment_ prefix)',
        'ticker': 'Primary stock ticker mentioned (uppercase)',
        'date': 'Date of post (YYYY-MM-DD)',
        'time': 'Time of post (HH:MM:SS)',
        'datetime': 'Full datetime (UTC)',
        'title': 'Post title (submissions only)',
        'text': 'Post body text',
        'score': 'Reddit score (upvotes - downvotes)',
        'num_comments': 'Number of comments (submissions only)',
        'author': 'Reddit username (pseudonymous)',
        'subreddit': 'Source subreddit',
        'tickers_all': 'List of all tickers mentioned',
        'ticker_count': 'Number of tickers mentioned',
        'post_type': 'Type: submission or comment'
    }
    
    dict_path = os.path.join(output_dir, 'data_dictionary.json')
    with open(dict_path, 'w') as f:
        json.dump(data_dict, f, indent=2)
    print(f"Saved data dictionary: {dict_path}")
    
    # Summary statistics
    summary = {
        'total_posts': len(df),
        'date_range': [str(df['date'].min()), str(df['date'].max())],
        'posts_with_ticker': int(df['ticker'].notna().sum()),
        'unique_tickers': int(df['ticker'].nunique()),
        'unique_authors': int(df['author'].nunique()),
        'submissions': int((df['post_type'] == 'submission').sum()),
        'comments': int((df['post_type'] == 'comment').sum()),
        'created_at': datetime.now().isoformat()
    }
    
    summary_path = os.path.join(output_dir, 'collection_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"Saved summary: {summary_path}")
    
    return summary

# Save final table (uncomment when ready)
# summary = save_post_level_table(post_level_table, config.PROCESSED_DATA_PATH)

## 9. Summary and Next Steps

### 9.1 Collection Summary

In [ ]:
# =============================================================================
# COLLECTION SUMMARY
# =============================================================================

print("""
╔══════════════════════════════════════════════════════════════╗
║           NOTEBOOK 1: DATA COLLECTION COMPLETE               ║
╚══════════════════════════════════════════════════════════════╝

OUTPUT FILES:
─────────────
• wsb_posts_raw.parquet      - Primary post-level dataset
• wsb_posts_sample.csv       - CSV sample for inspection
• data_dictionary.json       - Variable definitions
• collection_summary.json    - Summary statistics

DATA SCHEMA:
────────────
[platform_post_id, ticker, date, time, text, score, author, subreddit]

NEXT STEPS:
───────────
→ Notebook 2: Text Processing & Sentiment Analysis
  - Clean text data
  - Apply FinBERT sentiment scoring
  - Aggregate to firm-day level

IMPORTANT NOTES:
────────────────
1. Verify data quality before proceeding
2. Check ticker coverage is sufficient (>15% recommended)
3. Review temporal gaps in data
4. Document any data collection issues

""")

In [ ]:
# =============================================================================
# ENVIRONMENT INFO FOR REPRODUCIBILITY
# =============================================================================

import sys
import platform

print("Environment Information:")
print(f"  Python: {sys.version}")
print(f"  Platform: {platform.platform()}")
print(f"  Pandas: {pd.__version__}")
print(f"  NumPy: {np.__version__}")
print(f"  Timestamp: {datetime.now().isoformat()}")